# Python JSON模块完整指南

## 概述
JSON（JavaScript Object Notation）是现代Web开发中最重要的数据交换格式。Python的`json`模块提供了完整的JSON处理能力，是后端开发的必备技能。

## 核心概念对比
- **JavaScript**: `JSON.parse()` 和 `JSON.stringify()`
- **Python**: `json.loads()` 和 `json.dumps()`

## 学习目标
1. 掌握JSON序列化与反序列化
2. 处理复杂数据结构
3. 生产级错误处理
4. 性能优化技巧
5. 实际项目应用场景


In [2]:
import json
import logging
from typing import Dict, List, Any, Optional, Union
from datetime import datetime
from decimal import Decimal
import traceback

# 配置日志（生产级实践）
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

print("JSON模块导入成功 ✅")


JSON模块导入成功 ✅


# 1. 基础操作 - 序列化与反序列化

## 核心方法对比表

| Python方法 | JavaScript等价 | 功能 |
|------------|---------------|------|
| `json.dumps()` | `JSON.stringify()` | Python对象 → JSON字符串 |
| `json.loads()` | `JSON.parse()` | JSON字符串 → Python对象 |
| `json.dump()` | 无直接等价 | Python对象 → JSON文件 |
| `json.load()` | 无直接等价 | JSON文件 → Python对象 |


### 1.1 基础序列化示例

In [5]:
def demonstrate_basic_serialization():
    """演示基础JSON序列化操作"""
    
    # Python数据结构
    user_data = {
        "id": 1001,
        "name": "张三",
        "email": "zhangsan@example.com",
        "active": True,
        "roles": ["user", "admin"],
        "metadata": {
            "last_login": "2024-01-15",
            "preferences": {
                "theme": "dark",
                "language": "zh-CN"
            }
        }
    }
    
    # Python对象 → JSON字符串
    json_string = json.dumps(user_data, ensure_ascii=False, indent=2)
    print("📤 序列化结果:")
    print(json_string)
    print(f"类型: {type(json_string)}")
    
    # JSON字符串 → Python对象
    parsed_data = json.loads(json_string)
    print(f"\n📥 反序列化结果:")
    print(f"用户名: {parsed_data['name']}")
    print(f"类型: {type(parsed_data)}")
    print(f"是否相等: {user_data == parsed_data}")
    
    return json_string, parsed_data

    
# 执行演示
json_str, parsed_obj = demonstrate_basic_serialization()


📤 序列化结果:
{
  "id": 1001,
  "name": "张三",
  "email": "zhangsan@example.com",
  "active": true,
  "roles": [
    "user",
    "admin"
  ],
  "metadata": {
    "last_login": "2024-01-15",
    "preferences": {
      "theme": "dark",
      "language": "zh-CN"
    }
  }
}
类型: <class 'str'>

📥 反序列化结果:
用户名: 张三
类型: <class 'dict'>
是否相等: True


### 1.2 文件操作示例

In [6]:
def demonstrate_file_operations():
    """演示JSON文件读写操作"""
    
    # 准备测试数据
    api_config = {
        "api_version": "v1",
        "endpoints": {
            "users": "/api/v1/users",
            "auth": "/api/v1/auth",
            "products": "/api/v1/products"
        },
        "rate_limits": {
            "per_minute": 100,
            "per_hour": 5000
        },
        "features": {
            "cache_enabled": True,
            "logging_level": "INFO"
        }
    }
    
    filename = "api_config.json"
    
    try:
        # 写入JSON文件
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(api_config, f, ensure_ascii=False, indent=2)
        logger.info(f"✅ 配置已写入 {filename}")
        
        # 读取JSON文件
        with open(filename, 'r', encoding='utf-8') as f:
            loaded_config = json.load(f)
        
        print("📁 从文件加载的配置:")
        print(f"API版本: {loaded_config['api_version']}")
        print(f"用户端点: {loaded_config['endpoints']['users']}")
        print(f"每分钟限制: {loaded_config['rate_limits']['per_minute']}")
        
        # 验证数据完整性
        assert api_config == loaded_config, "数据不匹配!"
        print("✅ 文件读写验证成功")
        
        return loaded_config
        
    except Exception as e:
        logger.error(f"❌ 文件操作失败: {e}")
        raise

# 执行文件操作演示
config = demonstrate_file_operations()


INFO:__main__:✅ 配置已写入 api_config.json


📁 从文件加载的配置:
API版本: v1
用户端点: /api/v1/users
每分钟限制: 100
✅ 文件读写验证成功


`1. loaded_config明明是with open作用于内的变量，为什么在with open外面还可以使用？`

Python中，变量的作用域是基于函数级别的，不是基于代码块级别的。所以在with语句中定义的变量在整个函数内都可以访问，直到函数结束。这与JavaScript的let/const块级作用域不同。

`2. assert是什么语法？什么作用？`

assert是断言语句，用于验证条件是否为真。如果条件为假，会抛出AssertionError异常。格式为assert 条件, "错误信息"。它常用于调试和测试（多用于单元测试中），确保代码行为符合预期。在生产环境中通常会被禁用。

# 2. 进阶特性 - 定制化处理

## 2.1 重要参数说明

| 参数 | 作用 | 生产环境建议 |
|------|------|-------------|
| `ensure_ascii=False` | 支持中文字符 | 必须设置 |
| `indent=2` | 美化输出格式 | 开发时使用 |
| `sort_keys=True` | 键排序 | API响应时使用 |
| `separators=(',', ':')` | 压缩格式 | 生产环境使用 |

`ensure_ascii=False`:意思是，‘是否确定要显示ascii的字符’？设置为False的话，就可以显示除了ascii字符以外的字符，比如中文字符。如果设置为True，中文字符显示不出来的。


### 2.2 自定义序列化器 - 处理特殊数据类型

In [7]:
class ProductionJSONEncoder(json.JSONEncoder):
    """生产级JSON编码器，处理Python特殊类型"""
    
    def default(self, obj):
        """自定义对象序列化逻辑"""
        if isinstance(obj, datetime):
            return obj.isoformat()
        elif isinstance(obj, Decimal):
            return float(obj)
        elif isinstance(obj, set):
            return list(obj)
        elif hasattr(obj, '__dict__'):
            # 处理自定义对象
            return obj.__dict__
        else:
            return super().default(obj)

# 演示自定义序列化
def demonstrate_custom_serialization():
    """演示处理复杂数据类型"""
    
    # 包含特殊类型的数据
    complex_data = {
        "order_id": "ORD-2024-001",
        "created_at": datetime.now(),
        "total_amount": Decimal("299.99"),
        "tags": {"urgent", "vip", "priority"},  # set类型
        "metadata": {
            "processing_time": 1.234,
            "status_codes": [200, 201, 204]
        }
    }
    
    try:
        # 使用自定义编码器
        json_result = json.dumps(
            complex_data, 
            cls=ProductionJSONEncoder,
            ensure_ascii=False,
            indent=2
        )
        
        print("🔧 自定义序列化结果:")
        print(json_result)
        
        # 验证反序列化
        restored_data = json.loads(json_result)
        print(f"\n✅ 订单ID: {restored_data['order_id']}")
        print(f"✅ 创建时间: {restored_data['created_at']}")
        print(f"✅ 总金额: {restored_data['total_amount']}")
        print(f"✅ 标签: {restored_data['tags']}")
        
        return json_result
        
    except Exception as e:
        logger.error(f"❌ 自定义序列化失败: {e}")
        raise

# 执行自定义序列化演示
custom_json = demonstrate_custom_serialization()


🔧 自定义序列化结果:
{
  "order_id": "ORD-2024-001",
  "created_at": "2025-06-07T13:04:00.400716",
  "total_amount": 299.99,
  "tags": [
    "vip",
    "urgent",
    "priority"
  ],
  "metadata": {
    "processing_time": 1.234,
    "status_codes": [
      200,
      201,
      204
    ]
  }
}

✅ 订单ID: ORD-2024-001
✅ 创建时间: 2025-06-07T13:04:00.400716
✅ 总金额: 299.99
✅ 标签: ['vip', 'urgent', 'priority']


# 3. 生产级错误处理

在后端开发中，健壮的错误处理是必须的。JSON操作可能遇到多种异常情况。


### 3.1 生产级JSON处理工具类

In [ ]:
class SafeJSONHandler:
    """生产级JSON处理工具类"""
    
    @staticmethod
    def safe_dumps(data: Any, **kwargs) -> Optional[str]:
        """安全的JSON序列化"""
        try:
            return json.dumps(
                data, 
                ensure_ascii=False,
                cls=ProductionJSONEncoder,
                **kwargs
            )
        except TypeError as e:
            logger.error(f"JSON序列化类型错误: {e}")
            logger.debug(f"数据内容: {data}")
            return None
        except ValueError as e:
            logger.error(f"JSON序列化值错误: {e}")
            return None
        except Exception as e:
            logger.error(f"JSON序列化未知错误: {e}")
            logger.debug(traceback.format_exc())
            return None
    
    @staticmethod
    def safe_loads(json_str: str) -> Optional[Dict[str, Any]]:
        """安全的JSON反序列化"""
        if not json_str or not isinstance(json_str, str):
            logger.warning("输入的JSON字符串为空或类型错误")
            return None
            
        try:
            return json.loads(json_str)
        except json.JSONDecodeError as e:
            logger.error(f"JSON解析错误: {e}")
            logger.debug(f"错误位置: 行{e.lineno}, 列{e.colno}")
            logger.debug(f"错误内容: {json_str[:100]}...")
            return None
        except Exception as e:
            logger.error(f"JSON解析未知错误: {e}")
            return None
    
    @staticmethod
    def safe_load_file(filepath: str) -> Optional[Dict[str, Any]]:
        """安全的JSON文件加载"""
        try:
            with open(filepath, 'r', encoding='utf-8') as f:
                return json.load(f)
        except FileNotFoundError:
            logger.error(f"JSON文件不存在: {filepath}")
            return None
        except PermissionError:
            logger.error(f"无权限访问文件: {filepath}")
            return None
        except json.JSONDecodeError as e:
            logger.error(f"JSON文件格式错误: {filepath}, {e}")
            return None
        except Exception as e:
            logger.error(f"加载JSON文件失败: {filepath}, {e}")
            return None

# 演示错误处理
def demonstrate_error_handling():
    """演示各种错误情况的处理"""
    
    handler = SafeJSONHandler()
    
    # 测试用例1: 正常数据
    valid_data = {"name": "测试", "value": 123}
    result1 = handler.safe_dumps(valid_data)
    print(f"✅ 正常数据处理: {result1 is not None}")
    
    # 测试用例2: 不可序列化的数据
    class NonSerializable:
        def __init__(self):
            self.func = lambda x: x  # 函数不可序列化
    
    invalid_data = {"obj": NonSerializable()}
    result2 = handler.safe_dumps(invalid_data)
    print(f"❌ 不可序列化数据: {result2 is None}")
    
    # 测试用例3: 损坏的JSON字符串
    broken_json = '{"name": "test", "value":}'  # 缺少值
    result3 = handler.safe_loads(broken_json)
    print(f"❌ 损坏的JSON: {result3 is None}")
    
    # 测试用例4: 空输入处理
    result4 = handler.safe_loads("")
    result5 = handler.safe_loads(None)
    print(f"❌ 空输入处理: {result4 is None and result5 is None}")
    
    print("\n🛡️ 错误处理演示完成")

# 执行错误处理演示
demonstrate_error_handling()


# 4. 实际应用场景

### 4.1 Web API 响应格式化

In [ ]:
# Web API 响应格式化 (FastAPI风格)
def create_api_response(data: Any, status: str = "success", message: str = "") -> str:
    """创建标准化的API响应"""
    
    response_data = {
        "status": status,
        "message": message,
        "timestamp": datetime.now().isoformat(),
        "data": data
    }
    
    if status == "error":
        response_data["error_details"] = {
            "code": "INTERNAL_ERROR",
            "trace_id": f"trace-{datetime.now().timestamp()}"
        }
    
    return json.dumps(
        response_data,
        cls=ProductionJSONEncoder,
        ensure_ascii=False,
        separators=(',', ':')  # 生产环境压缩格式
    )

# 演示API响应
def demonstrate_api_responses():
    """演示不同类型的API响应"""
    
    # 成功响应
    user_list = [
        {"id": 1, "name": "张三", "email": "zhangsan@example.com"},
        {"id": 2, "name": "李四", "email": "lisi@example.com"}
    ]
    
    success_response = create_api_response(
        data=user_list,
        message="用户列表获取成功"
    )
    print("✅ 成功响应:")
    print(success_response)
    
    # 错误响应
    error_response = create_api_response(
        data=None,
        status="error",
        message="用户不存在"
    )
    print(f"\n❌ 错误响应:")
    print(error_response)
    
    return success_response, error_response

# 执行API响应演示
api_responses = demonstrate_api_responses()


### 4.2 配置文件管理

In [ ]:
class ConfigManager:
    """生产级配置管理器"""
    
    def __init__(self, config_path: str = "app_config.json"):
        self.config_path = config_path
        self.config = self._load_default_config()
        self._load_from_file()
    
    def _load_default_config(self) -> Dict[str, Any]:
        """默认配置"""
        return {
            "database": {
                "host": "localhost",
                "port": 5432,
                "name": "myapp",
                "pool_size": 10
            },
            "redis": {
                "host": "localhost",
                "port": 6379,
                "db": 0
            },
            "logging": {
                "level": "INFO",
                "format": "%(asctime)s - %(name)s - %(levelname)s - %(message)s"
            },
            "features": {
                "enable_cache": True,
                "enable_metrics": True,
                "debug_mode": False
            }
        }
    
    def _load_from_file(self) -> None:
        """从文件加载配置"""
        handler = SafeJSONHandler()
        file_config = handler.safe_load_file(self.config_path)
        
        if file_config:
            self._merge_config(file_config)
            logger.info(f"配置已从 {self.config_path} 加载")
        else:
            self.save_config()  # 创建默认配置文件
            logger.info(f"已创建默认配置文件: {self.config_path}")
    
    def _merge_config(self, new_config: Dict[str, Any]) -> None:
        """深度合并配置"""
        def deep_merge(base: dict, update: dict) -> dict:
            for key, value in update.items():
                if key in base and isinstance(base[key], dict) and isinstance(value, dict):
                    deep_merge(base[key], value)
                else:
                    base[key] = value
            return base
        
        deep_merge(self.config, new_config)
    
    def get(self, key_path: str, default=None):
        """获取配置值 (支持点分隔路径)"""
        keys = key_path.split('.')
        value = self.config
        
        for key in keys:
            if isinstance(value, dict) and key in value:
                value = value[key]
            else:
                return default
        
        return value
    
    def set(self, key_path: str, value: Any) -> None:
        """设置配置值"""
        keys = key_path.split('.')
        config = self.config
        
        for key in keys[:-1]:
            if key not in config:
                config[key] = {}
            config = config[key]
        
        config[keys[-1]] = value
        logger.info(f"配置已更新: {key_path} = {value}")
        
    def save_config(self) -> bool:
        """保存配置到文件"""
        handler = SafeJSONHandler()
        json_data = handler.safe_dumps(self.config, indent=2)
        
        if json_data:
            try:
                with open(self.config_path, 'w', encoding='utf-8') as f:
                    f.write(json_data)
                logger.info(f"配置已保存到 {self.config_path}")
                return True
            except Exception as e:
                logger.error(f"保存配置失败: {e}")
                return False
        return False

# 演示配置管理
def demonstrate_config_management():
    """演示配置管理系统"""
    
    config_mgr = ConfigManager("demo_config.json")
    
    # 读取配置
    db_host = config_mgr.get("database.host")
    cache_enabled = config_mgr.get("features.enable_cache")
    
    print("📋 当前配置:")
    print(f"数据库主机: {db_host}")
    print(f"缓存启用: {cache_enabled}")
    
    # 修改配置
    config_mgr.set("database.host", "production-db.example.com")
    config_mgr.set("features.debug_mode", True)
    
    # 保存配置
    success = config_mgr.save_config()
    print(f"\n💾 配置保存: {'成功' if success else '失败'}")
    
    return config_mgr

# 执行配置管理演示
config_manager = demonstrate_config_management()


### 4.3 数据传输与存储

In [ ]:
def demonstrate_data_transfer():
    """演示JSON在数据传输中的应用"""
    
    # 模拟数据库记录序列化
    database_record = {
        "id": 12345,
        "user_id": 9876,
        "created_at": datetime.now(),
        "updated_at": datetime.now(),
        "data": {
            "preferences": {"theme": "dark", "notifications": True},
            "statistics": {"login_count": 42, "last_active": datetime.now()}
        },
        "tags": {"premium", "verified"},
        "balance": Decimal("1299.99")
    }
    
    # 序列化用于存储
    serialized = json.dumps(database_record, cls=ProductionJSONEncoder, separators=(',', ':'))
    print("💾 数据存储格式:")
    print(f"大小: {len(serialized)} 字节")
    
    # 反序列化恢复数据
    restored = json.loads(serialized)
    print(f"✅ 数据恢复: 用户ID {restored['user_id']}, 余额 {restored['balance']}")
    
    return serialized


### 4.4 日志结构化

In [ ]:
def demonstrate_structured_logging():
    """演示结构化日志的JSON应用"""
    
    import sys
    
    def create_log_entry(level: str, message: str, **context):
        """创建结构化日志条目"""
        log_entry = {
            "timestamp": datetime.now().isoformat(),
            "level": level.upper(),
            "message": message,
            "context": context,
            "system": {
                "python_version": sys.version.split()[0],
                "platform": sys.platform
            }
        }
        return json.dumps(log_entry, cls=ProductionJSONEncoder, separators=(',', ':'))
    
    # 不同级别的日志示例
    info_log = create_log_entry("info", "用户登录成功", user_id=123, ip="192.168.1.1")
    error_log = create_log_entry("error", "数据库连接失败", 
                                database="users", attempt=3, error_code="CONN_TIMEOUT")
    
    print("📝 结构化日志示例:")
    print("INFO:", info_log)
    print("ERROR:", error_log)
    
    return info_log, error_log

# 执行演示
data_transfer_demo = demonstrate_data_transfer()
log_demo = demonstrate_structured_logging()


# 5. 性能优化与最佳实践

### 5.1 性能对比测试

In [ ]:
# 性能优化演示
import time
from typing import Generator

def performance_comparison():
    """性能对比测试"""
    
    # 测试数据
    large_data = {
        "users": [
            {
                "id": i,
                "name": f"用户{i}",
                "email": f"user{i}@example.com",
                "metadata": {
                    "created_at": datetime.now().isoformat(),
                    "tags": [f"tag{j}" for j in range(5)],
                    "settings": {"theme": "dark", "lang": "zh"}
                }
            } for i in range(1000)  # 1000个用户
        ]
    }
    
    print("📊 性能对比测试 (1000用户数据)")
    print("-" * 50)
    
    # 测试1: 标准序列化
    start_time = time.time()
    json_std = json.dumps(large_data, ensure_ascii=False)
    std_time = time.time() - start_time
    print(f"标准序列化: {std_time:.4f}秒, 大小: {len(json_std):,}字符")
    
    # 测试2: 压缩序列化
    start_time = time.time()
    json_compact = json.dumps(large_data, ensure_ascii=False, separators=(',', ':'))
    compact_time = time.time() - start_time
    print(f"压缩序列化: {compact_time:.4f}秒, 大小: {len(json_compact):,}字符")
    
    # 测试3: 自定义编码器
    start_time = time.time()
    json_custom = json.dumps(large_data, cls=ProductionJSONEncoder, ensure_ascii=False)
    custom_time = time.time() - start_time
    print(f"自定义编码: {custom_time:.4f}秒, 大小: {len(json_custom):,}字符")
    
    # 测试反序列化性能
    start_time = time.time()
    parsed_data = json.loads(json_compact)
    parse_time = time.time() - start_time
    print(f"反序列化: {parse_time:.4f}秒")
    
    # 压缩效率
    compression_ratio = (len(json_std) - len(json_compact)) / len(json_std) * 100
    print(f"\n💡 压缩效率: {compression_ratio:.1f}% 减少")
    
    return {
        "std_time": std_time,
        "compact_time": compact_time,
        "custom_time": custom_time,
        "parse_time": parse_time,
        "compression_ratio": compression_ratio
    }

# 执行性能测试
perf_results = performance_comparison()


### 5.2 内存优化技巧  

In [ ]:
def demonstrate_memory_optimization():
    """演示JSON处理的内存优化技巧"""
    
    print("💡 内存优化技巧演示:")
    print("-" * 30)
    
    # 技巧1: 使用生成器处理大量数据
    def json_generator(data_source):
        """使用生成器逐条处理JSON数据"""
        for item in data_source:
            yield json.dumps(item, separators=(',', ':'))
    
    # 技巧2: 流式写入大文件
    def write_large_json_stream(filename: str, data_count: int = 1000):
        """流式写入大型JSON文件"""
        with open(filename, 'w', encoding='utf-8') as f:
            f.write('[')
            for i in range(data_count):
                if i > 0:
                    f.write(',')
                item = {"id": i, "data": f"item_{i}", "timestamp": datetime.now().isoformat()}
                json.dump(item, f, separators=(',', ':'))
                
                # 每100条刷新一次缓冲区
                if i % 100 == 0:
                    f.flush()
            f.write(']')
    
    # 技巧3: 分块读取大文件
    def read_json_chunks(filename: str, chunk_size: int = 1024):
        """分块读取JSON文件"""
        with open(filename, 'r', encoding='utf-8') as f:
            while True:
                chunk = f.read(chunk_size)
                if not chunk:
                    break
                yield chunk
    
    # 演示流式写入
    test_file = "large_data.json"
    write_large_json_stream(test_file, 100)
    print(f"✅ 已创建测试文件: {test_file}")
    
    # 演示内存效率
    import os
    file_size = os.path.getsize(test_file)
    print(f"📊 文件大小: {file_size:,} 字节")
    
    return test_file

### 5.3 生产环境建议

In [ ]:
def production_best_practices():
    """生产环境JSON处理最佳实践"""
    
    recommendations = {
        "性能优化": [
            "使用 separators=(',', ':') 压缩输出",
            "大数据量时考虑使用 orjson 库",
            "避免深层嵌套结构(建议<10层)",
            "缓存频繁使用的JSON模式"
        ],
        "安全考虑": [
            "限制JSON文件大小(建议<10MB)",
            "验证输入数据结构",
            "使用JSON Schema进行验证",
            "避免执行动态JSON内容"
        ],
        "错误处理": [
            "始终捕获JSONDecodeError异常",
            "记录详细的错误日志",
            "提供降级处理方案",
            "设置合理的超时时间"
        ],
        "监控指标": [
            "JSON序列化/反序列化时间",
            "内存使用峰值",
            "错误率统计",
            "文件大小趋势"
        ]
    }
    
    print("🏭 生产环境最佳实践:")
    print("=" * 40)
    
    for category, practices in recommendations.items():
        print(f"\n📋 {category}:")
        for practice in practices:
            print(f"  • {practice}")
    
    # 示例：生产级JSON处理模板
    def production_json_handler(data, operation="dumps"):
        """生产级JSON处理模板"""
        import time
        start_time = time.time()
        
        try:
            if operation == "dumps":
                result = json.dumps(data, cls=ProductionJSONEncoder, 
                                  ensure_ascii=False, separators=(',', ':'))
            else:
                result = json.loads(data)
                
            processing_time = time.time() - start_time
            logger.info(f"JSON {operation} 成功, 耗时: {processing_time:.4f}秒")
            return result
            
        except Exception as e:
            processing_time = time.time() - start_time
            logger.error(f"JSON {operation} 失败: {e}, 耗时: {processing_time:.4f}秒")
            return None
    
    print(f"\n✅ 生产级处理模板已定义")
    return recommendations

# 执行优化演示
memory_demo = demonstrate_memory_optimization()
best_practices = production_best_practices()


# 6. 生产环境最佳实践总结

## ✅ 必须遵循的原则

### 1. 字符编码
```python
# ✅ 正确 - 支持中文
json.dumps(data, ensure_ascii=False)

# ❌ 错误 - 中文会被转义
json.dumps(data, ensure_ascii=True)  # 默认值
```

### 2. 错误处理
```python
# ✅ 正确 - 完整错误处理
try:
    data = json.loads(json_string)
except json.JSONDecodeError as e:
    logger.error(f"JSON解析错误: {e}")
    return None

# ❌ 错误 - 没有错误处理
data = json.loads(json_string)  # 可能崩溃
```

### 3. 性能优化
```python
# ✅ 生产环境 - 压缩格式
json.dumps(data, separators=(',', ':'))

# ✅ 开发环境 - 美化格式  
json.dumps(data, indent=2)
```

### 4. 类型安全
```python
# ✅ 正确 - 使用类型注解
def safe_loads(json_str: str) -> Optional[Dict[str, Any]]:
    pass

# ❌ 错误 - 没有类型提示
def parse_json(data):
    pass
```


# 附录：JSON模块完整API参考

## 核心函数详解

### 1. json.dumps(obj, **kwargs)
**功能**: Python对象 → JSON字符串
**常用参数**:
- `ensure_ascii=False`: 支持非ASCII字符(中文)
- `indent=None`: 缩进美化(开发用)，None为紧凑(生产用)
- `separators=None`: 分隔符，(',', ':')为最紧凑
- `sort_keys=False`: 是否排序键
- `cls=None`: 自定义编码器类
- `default=None`: 默认序列化函数

### 2. json.loads(s, **kwargs)
**功能**: JSON字符串 → Python对象
**常用参数**:
- `cls=None`: 自定义解码器类
- `object_hook=None`: 对象转换钩子
- `parse_float=None`: 浮点数解析器
- `parse_int=None`: 整数解析器

### 3. json.dump(obj, fp, **kwargs)
**功能**: Python对象 → JSON文件
**参数**: 同dumps()，额外需要文件对象fp

### 4. json.load(fp, **kwargs)
**功能**: JSON文件 → Python对象
**参数**: 同loads()，额外需要文件对象fp

## 重要异常类型

- `json.JSONDecodeError`: JSON解析错误
- `TypeError`: 对象不可序列化
- `ValueError`: 值错误(如循环引用)

## 数据类型映射

| Python | JSON | 说明 |
|--------|------|------|
| dict | object | 字典对象 |
| list, tuple | array | 数组 |
| str | string | 字符串 |
| int, float | number | 数值 |
| True | true | 布尔真 |
| False | false | 布尔假 |
| None | null | 空值 |


In [1]:
# 7. 实战练习题

def create_practice_exercises():
    """创建JSON练习题"""
    
    exercises = [
        {
            "level": "初级",
            "title": "API响应格式化",
            "description": "创建一个函数，接收用户数据列表，返回标准JSON API响应",
            "requirements": [
                "包含status、message、data字段",
                "支持中文字符", 
                "添加时间戳",
                "处理空数据情况"
            ],
            "sample_input": [
                {"id": 1, "name": "张三"},
                {"id": 2, "name": "李四"}
            ]
        },
        {
            "level": "中级", 
            "title": "配置文件合并",
            "description": "实现配置文件的深度合并功能",
            "requirements": [
                "支持嵌套字典合并",
                "保留原有配置",
                "添加新配置项",
                "提供冲突处理策略"
            ],
            "sample_input": {
                "base": {"db": {"host": "localhost", "port": 5432}},
                "update": {"db": {"port": 3306}, "cache": {"enabled": True}}
            }
        },
        {
            "level": "高级",
            "title": "JSON流处理器", 
            "description": "处理大型JSON文件，实现流式读取和处理",
            "requirements": [
                "支持大文件处理(>100MB)",
                "内存使用优化",
                "错误恢复机制", 
                "进度跟踪"
            ],
            "challenge": "在1GB的JSON文件中查找特定数据，内存使用不超过100MB"
        }
    ]
    
    print("🎯 JSON模块实战练习题")
    print("=" * 50)
    
    for i, exercise in enumerate(exercises, 1):
        print(f"\n【练习{i}】{exercise['title']} - {exercise['level']}")
        print(f"描述: {exercise['description']}")
        print("要求:")
        for req in exercise['requirements']:
            print(f"  • {req}")
        
        if 'sample_input' in exercise:
            print(f"示例输入: {exercise['sample_input']}")
        if 'challenge' in exercise:
            print(f"挑战: {exercise['challenge']}")
    
    print(f"\n💪 选择一个练习开始编码吧！")
    return exercises

# 生成练习题
practice_exercises = create_practice_exercises()


🎯 JSON模块实战练习题

【练习1】API响应格式化 - 初级
描述: 创建一个函数，接收用户数据列表，返回标准JSON API响应
要求:
  • 包含status、message、data字段
  • 支持中文字符
  • 添加时间戳
  • 处理空数据情况
示例输入: [{'id': 1, 'name': '张三'}, {'id': 2, 'name': '李四'}]

【练习2】配置文件合并 - 中级
描述: 实现配置文件的深度合并功能
要求:
  • 支持嵌套字典合并
  • 保留原有配置
  • 添加新配置项
  • 提供冲突处理策略
示例输入: {'base': {'db': {'host': 'localhost', 'port': 5432}}, 'update': {'db': {'port': 3306}, 'cache': {'enabled': True}}}

【练习3】JSON流处理器 - 高级
描述: 处理大型JSON文件，实现流式读取和处理
要求:
  • 支持大文件处理(>100MB)
  • 内存使用优化
  • 错误恢复机制
  • 进度跟踪
挑战: 在1GB的JSON文件中查找特定数据，内存使用不超过100MB

💪 选择一个练习开始编码吧！


# 总结

## 🎓 你现在掌握了什么

1. **基础操作**: `dumps()`, `loads()`, `dump()`, `load()`
2. **生产级错误处理**: 完整的异常捕获和日志记录
3. **自定义序列化**: 处理datetime、Decimal等特殊类型
4. **性能优化**: 压缩格式、内存优化技巧
5. **实际应用**: API响应、配置管理、数据传输

## 🚀 下一步学习建议

- **进阶**: 学习`orjson`库（更快的JSON处理）
- **实践**: 在FastAPI项目中应用JSON技能
- **扩展**: 研究JSON Schema验证
- **优化**: 学习JSON压缩算法

## 💡 记住这些关键点

- 始终使用`ensure_ascii=False`支持中文
- 生产环境必须有错误处理
- 大数据量时考虑性能优化
- 使用类型注解提高代码质量
- 日志记录帮助调试问题

**继续加油，你的后端之路越来越精彩！** 🎉
